<a href="https://colab.research.google.com/github/PabloParadaSouto/Automatica/blob/master/ACTOR-CRITIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import gym

In [3]:
# Definir la red neuronal para el actor
class Actor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, output_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

# Definir la red neuronal para el crítico
class Critic(nn.Module):
    def __init__(self, input_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Implementar el algoritmo ACTOR-Critic
def actor_critic(env_name, max_episodes):
    env = gym.make(env_name)
    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n

    actor = Actor(input_dim, output_dim)
    critic = Critic(input_dim)
    actor_optimizer = optim.Adam(actor.parameters(), lr=0.001)
    critic_optimizer = optim.Adam(critic.parameters(), lr=0.001)

    for episode in range(max_episodes):
        state = env.reset()
        rewards = []
        log_probs = []

        while True:
            action_probs = actor(torch.FloatTensor(state))
            action = torch.multinomial(action_probs, 1).item()
            log_probs.append(torch.log(action_probs[action]))

            next_state, reward, done, _ = env.step(action)
            rewards.append(reward)

            if done:
                Q_values = []
                Q_value = 0
                for r in rewards[::-1]:
                    Q_value = r + 0.99 * Q_value
                    Q_values.insert(0, Q_value)

                Q_values = torch.tensor(Q_values)
                log_probs = torch.stack(log_probs)
                values = critic(torch.FloatTensor(state))
                advantages = Q_values - values

                actor_loss = -torch.mean(log_probs * advantages.detach())
                critic_loss = torch.mean(advantages.pow(2))

                actor_optimizer.zero_grad()
                actor_loss.backward()
                actor_optimizer.step()

                critic_optimizer.zero_grad()
                critic_loss.backward()
                critic_optimizer.step()

                break

            state = next_state

# Ejecutar el algoritmo
if __name__ == "__main__":
    actor_critic('CartPole-v1', max_episodes=1000)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym